In [7]:
import pickle

import openmm
from openmm import unit

from openmmtools.states import ThermodynamicState, CompoundThermodynamicState
from openmmtools.constants import kB

from perses.annihilation.lambda_protocol import RESTCapableRelativeAlchemicalState

In [3]:
import numpy as np
n_states = 12
np.linspace(0.,1.,n_states)

array([0.        , 0.09090909, 0.18181818, 0.27272727, 0.36363636,
       0.45454545, 0.54545455, 0.63636364, 0.72727273, 0.81818182,
       0.90909091, 1.        ])

## Test on ala dipeptide

In [13]:
# htf = solvent_delivery.get_apo_htf()
with open("atp_solvent_scale_region.pickle", "rb") as f:
    htf = pickle.load(f)
hybrid_system = htf.hybrid_system
hybrid_positions = htf.hybrid_positions
box_vectors = hybrid_system.getDefaultPeriodicBoxVectors()


In [14]:
T_min = 300 * unit.kelvin
T_max = 600 * unit.kelvin

# Create thermodynamic state
lambda_zero_alchemical_state = RESTCapableRelativeAlchemicalState.from_system(hybrid_system)
thermostate = ThermodynamicState(hybrid_system, temperature=T_min)
compound_thermodynamic_state_0 = CompoundThermodynamicState(thermostate,
                                                          composable_states=[lambda_zero_alchemical_state])
compound_thermodynamic_state_1 = CompoundThermodynamicState(thermostate,
                                                          composable_states=[lambda_zero_alchemical_state])

# Set alchemical parameters
beta_0 = 1 / (kB * T_min)
beta_m = 1 / (kB * T_max)
global_lambda = 0
compound_thermodynamic_state_0.set_alchemical_parameters(global_lambda, beta_0, beta_m)
global_lambda = 1
compound_thermodynamic_state_1.set_alchemical_parameters(global_lambda, beta_0, beta_m)


In [21]:
forces

{'CustomBondForce': <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b79c5bb7ba0> >,
 'CustomAngleForce': <openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b79c60a2f00> >,
 'CustomTorsionForce': <openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b79c60a2e70> >,
 'CustomNonbondedForce_electrostatics': <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2b79c7635fc0> >,
 'CustomNonbondedForce_sterics': <openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2b79c7635d20> >,
 'CustomBondForce_exceptions': <openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b79c7635de0> >,
 'NonbondedForce': <openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b79c7635e70> >,
 'A

In [39]:
unsampled_endstates = []
for master_lambda, endstate in zip([0., 1.], [compound_thermodynamic_state_0, compound_thermodynamic_state_1]):
    dispersion_system = endstate.get_system()
    energy_unit = unit.kilocalories_per_mole
    # Find the NonbondedForce (there must be only one)
    forces = { force.getName() : force for force in dispersion_system.getForces() }
    # Set NonbondedForce to use LJPME
    forces['NonbondedForce'].setNonbondedMethod(openmm.NonbondedForce.LJPME)
    # Set tight PME tolerance
    TIGHT_PME_TOLERANCE = 1.0e-5
    forces['NonbondedForce'].setEwaldErrorTolerance(TIGHT_PME_TOLERANCE)
    # Move alchemical LJ sites from CustomNonbondedForce back to NonbondedForce
    for particle_index in range(forces['NonbondedForce'].getNumParticles()):
        charge, sigma, epsilon = forces['NonbondedForce'].getParticleParameters(particle_index)
#         params_electrostatics = forces['CustomNonbondedForce_electrostatics'].getParticleParameters(particle_index)
#         charge_old, charge_new = params_electrostatics[-2:]
        params_sterics = forces['CustomNonbondedForce_sterics'].getParticleParameters(particle_index)
        sigma_old, sigma_new, epsilon_old, epsilon_new = params_sterics[-4:]
#         charge_final = (1 - master_lambda)* charge_old + master_lambda * charge_new
        sigma_final = (1 - master_lambda)* sigma_old + master_lambda * sigma_new
        epsilon_final = (1 - master_lambda)* epsilon_old + master_lambda * epsilon_new
#         print(particle_index, charge.value_in_unit_system(unit.md_unit_system), charge_old, charge_new, charge_other)
#         assert charge.value_in_unit_system(unit.md_unit_system) == charge_other
        
#         if (epsilon/energy_unit == 0.0) and ((epsilonA > 0.0) or (epsilonB > 0.0)):
#             sigma = (1-master_lambda)*sigmaA + master_lambda*sigmaB
#             epsilon = (1-master_lambda)*epsilonA + master_lambda*epsilonB
        forces['NonbondedForce'].setParticleParameters(particle_index, charge, sigma_final, epsilon_final)
    # Delete the CustomNonbondedForce since we have moved all alchemical particles out of it
    for force_index, force in enumerate(list(dispersion_system.getForces())):
#         if force.getName() == 'CustomNonbondedForce_electrostatics':
#             electrostatics_force_index = force_index
        if force.getName() == 'CustomNonbondedForce_sterics':
            sterics_force_index = force_index
    dispersion_system.removeForce(electrostatics_force_index)
    dispersion_system.removeForce(sterics_force_index)
    # Set all parameters to master lambda
    for force_index, force in enumerate(list(dispersion_system.getForces())):
        if hasattr(force, 'getNumGlobalParameters'):
            for parameter_index in range(force.getNumGlobalParameters()):
                parameter_name = force.getGlobalParameterName(parameter_index)
                if parameter_name[0:17] == 'lambda_alchemical':
                    if 'old' in parameter_name:
                        force.setGlobalParameterDefaultValue(parameter_index, 1 - master_lambda)
                    elif 'new' in parameter_name:
                        force.setGlobalParameterDefaultValue(parameter_index, master_lambda)
                        
    # Store the unsampled endstate
    unsampled_endstates.append(ThermodynamicState(dispersion_system, temperature = endstate.temperature))

### For each endstate, check that the steric energy in the CompoundThermodynamicState matches that in the ThermodynamicState

In [41]:
# Create context

# Retrieve energy components

<bound method ThermodynamicState.reduced_potential of <openmmtools.states.CompoundThermodynamicState object at 0x2b79c5bbab20>>

## Test on bnbs